# Python example for CMK-encryption in Azure AI Search

This notebook provides sample script for [adding customer-managed key (CMK) encryption](https://learn.microsoft.com/azure/search/search-security-manage-encryption-keys) to objects on Azure AI Search.

## Prerequisites


- [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal)
- [Azure Key Vault]()
- [Azure Storage](https://learn.microsoft.com/azure/storage/common/storage-account-create) or [Azure Log Analytics](https://learn.microsoft.com/azure/azure-monitor/logs/quick-create-workspace?tabs=azure-portal) for data retention of audit logs.


## Sign in to Azure

You might not need this step, but if downstream connections fail with a 401 during indexer pipeline execution, it could be because you're using the wrong tenant or subscription. You can avoid this issue by signing in from the command line, explicitly setting the tenant ID and choosing the right subscription.

This section assumes you have the [Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively).

1. Open a command line prompt.

1. Run this command to get a list of Azure tenants: `az account tenant list`

1. If you have multiple tenants, set the tenant: `az login --tenant <YOUR-TENANT_ID>`

If you have multiple subscriptions, a list is provided so that you can select one.

## Create a virtual environment in Visual Studio Code

Create a virtual environment so that you can install the dependencies in isolation.

1. In Visual Studio Code, open the folder containing tutorial-rag.ipynb.

1. Press Ctrl-shift-P to open the command palette, search for "Python: Create Environment", and then select `Venv` to create a virtual environment in the current workspace.

1. Select Tutorial-RAG\tutorial-rag-requirements.txt for the dependencies.

It takes several minutes to create the environment. When the environment is ready, continue to the next step.

## Install packages

In [ ]:
! pip install python-dotenv
! pip install azure-core
! pip install azure-search-documents
! pip install azure-storage-blob
! pip install azure-identity
! pip install openai
! pip install aiohttp

## CREATE INDEX

In [11]:
# Set variables
import os
from dotenv import load_dotenv
load_dotenv(override=True)

AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_KEY_VAULT_NAME = os.getenv("AZURE_KEY_VAULT_NAME")
AZURE_KEY_VAULT_URI = os.getenv("AZURE_KEY_VAULT_URI")
AZURE_KEY_VAULT_VERSION = os.getenv("AZURE_KEY_VAULT_VERSION")

In [12]:
print("key_name:", AZURE_KEY_VAULT_NAME)
print("key_version:", AZURE_KEY_VAULT_VERSION)
print("vault_uri:", AZURE_KEY_VAULT_URI)

key_name: key1
key_version: 77b89cfe1b954113b5d7fb91e304cacf
vault_uri: https://kor-ai-kv-13a-2.vault.azure.net/


In [13]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SearchResourceEncryptionKey
)
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

# Create a search index  
index_name = "test-cmk-index"
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)  
fields = [
        SimpleField(name="Id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="Description", type=SearchFieldDataType.String)
    ]

scoring_profiles = []
suggester = []

encryption_key = SearchResourceEncryptionKey(
    key_name=AZURE_KEY_VAULT_NAME,
    key_version=AZURE_KEY_VAULT_VERSION,
    vault_uri=AZURE_KEY_VAULT_URI
)

# Create the search index=
index = SearchIndex(name=index_name, fields=fields, encryption_key=encryption_key)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 test-cmk-index created


## GET INDEX DEFINITION


In [14]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SearchResourceEncryptionKey
)
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

# Create a search index  
index_name = "test-cmk-index"
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)  
fields = [
        SimpleField(name="Id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="Description", type=SearchFieldDataType.String)
    ]

scoring_profiles = []
suggester = []
encryption_key = SearchResourceEncryptionKey(
    key_name=AZURE_KEY_VAULT_NAME,
    key_version=AZURE_KEY_VAULT_VERSION,
    vault_uri=AZURE_KEY_VAULT_URI
)

# Create the search index=
index = SearchIndex(name=index_name, fields=fields, encryption_key=encryption_key)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 test-cmk-index created


In [15]:
index_name = "test-cmk-index"
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)  

result = index_client.get_index(index_name)  
print(f"{result}")  

{'additional_properties': {}, 'name': 'test-cmk-index', 'fields': [<azure.search.documents.indexes.models._index.SearchField object at 0x000001C2BBCE6F50>, <azure.search.documents.indexes.models._index.SearchField object at 0x000001C2BBCE5C50>], 'scoring_profiles': [], 'default_scoring_profile': None, 'cors_options': None, 'suggesters': [], 'analyzers': None, 'tokenizers': None, 'token_filters': [], 'char_filters': [], 'normalizers': [], 'encryption_key': <azure.search.documents.indexes.models._models.SearchResourceEncryptionKey object at 0x000001C2BBCE6550>, 'similarity': <azure.search.documents.indexes._generated.models._models_py3.BM25SimilarityAlgorithm object at 0x000001C2BBCF2D50>, 'semantic_search': None, 'vector_search': None, 'permission_filter_option': None, 'e_tag': '"0x8DDBAE194916ADD"'}


## LOAD INDEX

In [16]:
from azure.search.documents import SearchClient

# Create a documents payload
documents = [
    {
    "@search.action": "upload",
    "Id": "1",
    "Description": "The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities."
    },
    {
    "@search.action": "upload",
    "Id": "2",
    "Description": "The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts."
    },
    {
    "@search.action": "upload",
    "Id": "3",
    "Description": "The hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services."
    },
    {
    "@search.action": "upload",
    "Id": "4",
    "Description": "The hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace."
    }
]

search_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE, index_name=index_name, credential=credential)
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

    index_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)

Upload of new document succeeded: True


## QUERY ENCRYPTED CONTENT

In [17]:
from azure.search.documents import SearchClient

query = "historic"  

search_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential, index_name=index_name)
  
results = search_client.search(  
    query_type='simple',
    search_text=query, 
    select=["Id", "Description"],
    include_total_count=True
    )
  
for result in results:  
    print(f"Score: {result['@search.score']}")
    print(f"Id: {result['Id']}")
    print(f"Description: {result['Description']}")


Score: 0.6130029
Id: 4
Description: The hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.
Score: 0.26286605
Id: 1
Description: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
